$ \newcommand{\bra}[1]{\langle #1|} $
$ \newcommand{\ket}[1]{|#1\rangle} $
$ \newcommand{\braket}[2]{\langle #1|#2\rangle} $
$ \newcommand{\ketbra}[2]{| #1\rangle \langle #2|} $
$ \newcommand{\tr}{{\rm tr}} $
$ \newcommand{\i}{{\color{blue} i}} $
$ \newcommand{\Hil}{{\cal H}} $
$ \newcommand{\V}{{\cal V}} $
$ \newcommand{\bn}{{\bf n}} $

# 2. Single-qubit driving

The pulse-level implementation of some of the most common native gates will be explored, including the physical meaning of qubit driving, the different techniques and features available in Qmio using the OpenPulse grammar, and illustrative examples of gate calibration and single-qubit operations based on Rabi oscillations.

## 2.1. Qubit pulse control

Now that we understand how a qubit system -such as a transmon- is physically described, we can move into action and learn how to interact with it to change its state and apply transformations equivalent to ideal quantum gates. The physical implementation of this driving depends on the specific hardware -for example, lasers for NV-centers- but in general, it consists of applying pulses to add energy to the system. For example, in supeconducting qubits like transmons, the control is performed by coupling the system to a microwave driving line, which feeds energy to the qubit in a manner analogous to a forced oscillator.


<br>
<div style="text-align: center;">
    <img src="./media/driving.png" width="420">
</div>
<br>


Interpreting these pulses as photons, the general idea is to apply them at the resonance frequency of the qubit, which corresponds to the transition frequency of $\ket{0}\leftrightarrow\ket{1}$. As a result, either excitation -via absorption- or relaxation -via stimulated emission- can happen, depending on the qubit's state: 

- If the qubit is in $\ket{0}$, only absorption can occur, exciting the system to $\ket{1}$. 

- If the qubit is in $\ket{1}$, the applied photons can't promote the system to a higher state because of the transmon's anharmonicity. In this case, the only possible effect is stimulated emission (similar to what occurs in lasers), which relaxes de qubit back to $\ket{0}$. 

When a continuous pulse of photons is applied, the populations of the states oscillate with time. This behaviour in known as **Rabi oscillations**. These oscillations are key to implementing quantum gates, because their period can be controlled by adjusting the pulse amplitude and duration. For example, by calibrating the pulse duration to completetly populate $\ket{1}$ starting from $\ket{0}$, one effectively implements an X gate.<br>
<br>
A natural question arises in this setup: **If the qubit is an oscillator, why does its energy sometimes decrease even though we are always adding energy?** The answer lies in the transmon's anharmonicity. If the qubit were a purely harmonic oscillator, driving it at the resonance frequency would lead to unbounded excitation, promoting the system indefinitely through higher energy states -just as in a classical forced harmonic oscillator. The anharmonicity of the transmon limits the population of the oscillator states, as only the $\ket{0}\leftrightarrow\ket{1}$ transition is allowed.<br>
<br>
It may not be still clear how the anharmonicity is responsible of this effect in transmons, but luckily there is a classical system that describes a similar behaviour: the **forced non-linear pendulum**. Bringing our attention to the classical pendulum, when the oscillations are small enough, we can model it as an harmonic oscillator with a resonance frequency $\left(g/L\right)$ but, when the oscillations become grow, the non-linear terms become relevan and the resonance frequency is shifted. If an oscillating force -pulse- is applied on the pendulum with at the resonance frequency of its harmonic regime, the energy of the system will be bounded -meaning that we will reach a maximum amplitude- because the frequency will be detuned by the non-linear terms.<br>
<br>

Although they are not physically equivalent systems, we can make the following analogy: when the pendulum has minimum potential energy (no amplitude) we say that it's on the state $\ket{0}$, and when it is in state $\ket{1}$ whe it has maximum aplitude. Obviously, for the classical pendulum there is an infinite number of energy eigenstates between those two states, but we can understand these intermidiate states as the superposition of $\ket{0}$ and $\ket{1}$. The observed amplitude from the oscillation can be interpreted as the mean value, which depends on the populations of both states. On the following video is a representation of this system drived by a constant amplitude pulse, keep in mind that this is the simulation of the classical ecuations for the forced non-linear pendulum:

<br>
<br>
<div style="text-align: center;">
  <video width="720" height="480" controls>
    <source src="./media/DrivenPendulum.mp4" type="video/mp4">
    Tu navegador no soporta el video.
  </video>
</div>
<br>
<br>

As you can see, the system starts with zero energy ($\ket{0}$) and, at first, the incoming pulse excites the system, promoting the state and increasing the mean energy. This is analogous to the state $\ket{1}$ being populated by absorption. After some time, we arrive to the maximum amplitude ($\ket{1}$), from where the mean energy begins to decrease again. This is analogous to the relaxation of the system by stimulated emission. Finally, the characteristic Rabi-like periodic behaviour appear as the pulse continues to being applied.   


## 2.2. Pulse-level quantum gates

## 2.3. Pulse waveforms

## 2.4. Qubit driving with OpenPulse

## 2.5. Gate calibration examples